In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
#from keras.layers import Input, Dense, Lambda
#from keras.models import Model
#from keras.losses import binary_crossentropy

In [141]:
#Loading in data frame
data=pd.read_csv('C:\\Users\\Danie\\OneDrive\\Documents\\_Uni\\Maths\\Year 4\\Data Science Toolkit\\PreProcessedData.csv')
#Creating list of film plots as a list of strings
plots=data.Plot

In [142]:
#BASIC TEXT EDITS
import re
#Switching out hyphons for spaces
plots=[str(plots[num]).replace('-',' ') for num in range(len(plots))]
# Removing punctuation
plots=[re.sub(r'[^\w\s]','', str(plots[num])) for num in range(len(plots))]
# Lowercasing the words
plots=[str(plots[num]).lower() for num in range(len(plots))]

In [143]:
#REMOVING STOP WORDS AND TOKENIZING
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#Define english stop words
stop_words = set(stopwords.words('english'))

#Define function for removing stop words + tokenizing
def stopntokenize(text):
    word_tokens = word_tokenize(text)
    new_text = []
    for w in word_tokens:
        if w not in stop_words:
            new_text.append(w)
    return(new_text)
    

In [144]:
#WORDREMOVER

#Defining function for removing any word not a Noun, Adjective or Verb
def wordremover(tokens):
    wordtypes = nltk.pos_tag(tokens)
    tokens_new=[]
    for i in range(len(tokens)):
        if wordtypes[i][1] in ['NN','NNP','JJ','JJR','JJS','VB','VBD','VBG','VBN','VBP','VBZ']:
            tokens_new.append(tokens[i])
    return(tokens_new)

In [145]:
#LEAMMATIZING AND STEMMING
from nltk.corpus import wordnet

#Defining function to reduce word down it's simplest form
def lemmanstem(tokens):
    new_tokens=[]
    for i in range(len(tokens)):
        if wordnet.morphy(tokens[i])==None:
            new_tokens.append(tokens[i])
        else:
            new_tokens.append(wordnet.morphy(tokens[i]))
    return(new_tokens)   

In [146]:
#NONE REMOVER

#Defining function to remove Nones
def noneremover(tokens):
    for i in range(len(tokens)):
        if tokens[i]==None:
            tokens.remove(tokens[i])

In [147]:
#STRING TO TOKENS
def tokenizer(text):
    token=stopntokenize(text)
    token=wordremover(token)
    token=lemmanstem(token)
    token=list(dict.fromkeys(token))
    noneremover(token)
    return(token)

In [148]:
M=1443
plots[M]

'the story of tracy edwards a 24 year old cook on charter boats who became the skipper of the first ever all female crew to enter the whitbread round the world race in 1989'

In [149]:
tokenizer(plots[M])

['story',
 'tracy',
 'year',
 'old',
 'cook',
 'charter',
 'become',
 'first',
 'female',
 'crew',
 'whitbread',
 'round',
 'world',
 'race']

In [150]:
#WORD REDUCTION 

#Calculating percentage reduction of words for plots and tokenized plots
x=[]
for i in range(4000):
    x.append(len(tokenizer(plots[i]))/len(plots[i]))

sum(x)/4000

0.07198069794144195

In [151]:
#DATA SIZE

#Calculating average number of words per tokenized plot
#i.e. number of words remaining after text preprocessing
x=[]
for i in range(4000):
    x.append(len(tokenizer(plots[i])))

sum(x)/4000

38.78775

In [157]:
#CREATING DATASET OF TOKENIZED PLOTS
token_plots=[]
for i in range(len(plots)):
    token_plots.append(tokenizer(plots[i]))

In [153]:
#CREATING DICTIONARY
import gensim

dictionary = gensim.corpora.Dictionary(token_plots)

count = 0
for k,v  in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break
        
#dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

0 aberdeen
1 agree
2 airline
3 alcoholic
4 along
5 ban
6 begging
7 car
8 clive
9 coke
10 collect


In [154]:
#FINDING TOP WORDS

#Creating dictionary of wordcounts
new_dict=dictionary.cfs
#Sorting by wordcount
new_dict2 = sorted(new_dict.items(), key=lambda x:x[1],reverse=True)
#Translating word index to word
ranked_words=[]
for i in range(len(new_dict2)):
    ranked_words.append(dictionary[new_dict2[i][0]])
#Top 10 words
ranked_words[:10]

['life', 'find', 'new', 'take', 'get', 'young', 'family', 'world', 'go', 'man']

In [155]:
#DEFINING SYNONYM REMOVER
def synonymremover(tokens):
    #Defining list of unneccesary synonyms
    synonyms=[]
    for i in range(len(tokens)):
        for j in range(len(tokens)):
            word1=tokens[i]
            word2=tokens[j]
            #Get list of what the word could mean
            list_of_words1=wordnet.synsets(word1)
            list_of_words2=wordnet.synsets(word2)
            #Check that word has a possible definition
            if not (list_of_words1==[] or list_of_words2==[]):
                #Define words as the first choice 
                word_1=list_of_words1[0]
                word_2=list_of_words2[0]
                #Caculate similarity
                similarity=(word_1).path_similarity(word_2)
                #If they are close enough, add less significant word to synonyms
                if similarity>0.3 and similarity !=1:
                    #Calculate significance by getting their rank from the ranked words
                    sig1 = ranked_words.index(word1)
                    sig2 = ranked_words.index(word2)
                    #Add whichever word is lower down in the ranking to synonyms
                    if sig1<sig2:
                        synonyms.append(tokens[j])
                    else:
                        synonyms.append(tokens[i])
    #Remove all unneccesary synonyms
    for word in tokens:
        if word in synonyms:
            tokens.remove(word)
    return(tokens)

In [159]:
#UPDATING TOKENIZED PLOTS
#Takes quite a while atm lol

#for i in range(len(token_plots)):
#    token_plots[i]=synonymremover(token_plots[i])

In [169]:
#Trying a simialr thing with the actor names

#Getting dataframe of all actors. These are the top 3 billed in the film, separated by a comma and a space
actors=data.Actors
#Defining function to remove spaces between forname and surname, then switch commas to spaces
def namestowords(names):
    new_names=names.replace(' ','')
    new_names1=new_names.replace(',',' ')
    return(new_names1)

actors=[namestowords(actors[num]) for num in range(len(actors))]

In [170]:
#Creating dataframe of tokenized actors
tokenized_actors=[]
for i in range(len(actors)):
    tokenized_actors.append(word_tokenize(actors[i]))

In [174]:
#Repeating steps to create a sorted dictionary as above
dictionary_act = gensim.corpora.Dictionary(tokenized_actors)
new_dict_act=dictionary_act.cfs
#Sorting by wordcount
new_dict_act_2 = sorted(new_dict_act.items(), key=lambda x:x[1],reverse=True)
#Translating word index to word
ranked_actors=[]
for i in range(len(new_dict_act_2)):
    ranked_actors.append(dictionary2[new_dict_act_2[i][0]])
ranked_actors.remove('.')
#Top 30 most billed actors
ranked_actors[:30]

['BruceWillis',
 'SamuelL.Jackson',
 'NicolasCage',
 'LiamNeeson',
 'MarkWahlberg',
 'EwanMcGregor',
 'RyanReynolds',
 'RobertDeNiro',
 'DwayneJohnson',
 'OwenWilson',
 'MattDamon',
 'JohnnyDepp',
 'NicoleKidman',
 'ScarlettJohansson',
 'MorganFreeman',
 'WoodyHarrelson',
 'ColinFarrell',
 'JulianneMoore',
 'SteveCarell',
 'CharlizeTheron',
 'HughJackman',
 'AdamSandler',
 'CateBlanchett',
 'JasonStatham',
 'ColinFirth',
 'BenAffleck',
 'BradPitt',
 'TomHanks',
 'GerardButler',
 'ChristianBale']